In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.applications import inception_v3
import tensorflow as tf

In [2]:
K.set_learning_phase(0)

In [3]:
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

W1222 12:38:45.405623 140519690528576 deprecation.py:506] From /home/cleanpegasus/.local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.0,
    'mixed4': 2.0,
    'mixed5':1.5,
}

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [6]:
layer_dict

{'input_1': <tensorflow.python.keras.engine.input_layer.InputLayer at 0x7fccfd1d6a50>,
 'conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7fccfd1d6c90>,
 'batch_normalization': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7fccfd1d6d50>,
 'activation': <tensorflow.python.keras.layers.core.Activation at 0x7fccfd1a5f50>,
 'conv2d_1': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7fccfd1a50d0>,
 'batch_normalization_1': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7fccfd112250>,
 'activation_1': <tensorflow.python.keras.layers.core.Activation at 0x7fccfd1f2210>,
 'conv2d_2': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7fccfd1f2450>,
 'batch_normalization_2': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x7fccfd0ec390>,
 'activation_2': <tensorflow.python.keras.layers.core.Activation at 0x7fccfd094fd0>,
 'max_pooling2d': <tensorflow.python.keras.layers.pooling.MaxPooling2D at

In [7]:
loss = K.variable(0.0)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2 : -2, 2: -2, :])) / scaling

In [8]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads = grads/K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]

'1.14.0'